In [33]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from mizani.formatters import percent_format
import os
from plotnine import *
import numpy as np
import sys
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from stargazer import stargazer
from statsmodels.tools.eval_measures import mse,rmse

In [45]:
# Read the csv and feed it into the variable xc
data = pd.read_csv("https://raw.githubusercontent.com/Iandrewburg/Assignment_1/main/morg-2014-emp.csv?token=GHSAT0AAAAAACHIAHJF2NYISGDFSEGNFEXSZNK4VWA")

# Filter dataset for healthcare practitioners
data = data.loc[((data["occ2012"] >= 3000) & (data["occ2012"] <= 3540))]

In [46]:
data.head()

,Unnamed: 0,hhid,intmonth,stfips,weight,earnwke,uhours,grade92,race,ethnic,...,ownchild,chldpres,prcitshp,state,ind02,occ2012,class,unionmme,unioncov,lfsr94
10,25,620910090003769,January,AL,3851.5450,1800.00,40,43,1,NaN,...,2,4,"Native, Born In US",63,Pharmacies and drug stores (44611),3050,"Private, For Profit",No,No,Employed-At Work
33,60,7087505071191,January,AL,3712.0546,1171.60,40,41,1,NaN,...,0,0,"Native, Born In US",63,Business support services (5614),3510,"Private, For Profit",No,No,Employed-At Work
37,77,97953206049188,January,AL,3414.1067,1730.76,60,44,1,NaN,...,0,0,"Native, Born In US",63,Home health care services (6216),3255,"Private, For Profit",No,No,Employed-Absent
42,89,130003560100156,January,AL,3296.7986,2884.61,60,45,1,NaN,...,2,10,"Native, Born In US",63,Offices of physicians (6211),3060,"Private, For Profit",No,No,Employed-At Work
71,166,42510310991895,January,AL,3322.0487,618.75,45,40,1,NaN,...,0,0,"Native, Born In US",63,Hospitals (622),3320,"Private, For Profit",No,No,Employed-At Work


In [47]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9208 entries, 10 to 149315
Data columns (total 23 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  9208 non-null   int64  
 1   hhid        9208 non-null   int64  
 2   intmonth    9208 non-null   object 
 3   stfips      9208 non-null   object 
 4   weight      9208 non-null   float64
 5   earnwke     9208 non-null   float64
 6   uhours      9208 non-null   int64  
 7   grade92     9208 non-null   int64  
 8   race        9208 non-null   int64  
 9   ethnic      586 non-null    float64
 10  age         9208 non-null   int64  
 11  sex         9208 non-null   int64  
 12  marital     9208 non-null   int64  
 13  ownchild    9208 non-null   int64  
 14  chldpres    9208 non-null   int64  
 15  prcitshp    9208 non-null   object 
 16  state       9208 non-null   object 
 17  ind02       9208 non-null   object 
 18  occ2012     9208 non-null   int64  
 19  class       9208 non-null   o

### EDA

In [48]:
data["sex"] = data["sex"].fillna("Missing")
data["earnwke"] = data["earnwke"].fillna("Missing")
data["uhours"] = data["uhours"].fillna("Missing")
data["lfsr94"] = data["lfsr94"].fillna("Missing")
data["marital"] = data["marital"].fillna("Missing")
data["grade92"] = data["grade92"].fillna("Missing")
data["age"] = data["age"].fillna("Missing")
data["chldpres"] = data["chldpres"].fillna("Missing")


In [56]:
# Frequency table for chldpres
freq_chldpres = data.groupby("chldpres").agg(frequency=("chldpres", "size"))
freq_chldpres["percent"] = round(freq_chldpres["frequency"] / sum(freq_marital["frequency"]) * 100, 3)
freq_chldpres["cumulative_percent"] = np.cumsum(freq_chldpres["percent"])
print("Frequency Table for chldpres:")
freq_chldpres

Frequency Table for chldpres:


,frequency,percent,cumulative_percent
chldpres,,,
0,5347,58.069,58.069
1,457,4.963,63.032
2,272,2.954,65.986
3,949,10.306,76.292
4,634,6.885,83.177
5,262,2.845,86.022
6,130,1.412,87.434
7,15,0.163,87.597
8,382,4.149,91.746


In [58]:
# Frequency table for lfsr94
freq_marital = data.groupby("marital").agg(frequency=("marital", "size"))
freq_marital["percent"] = round(freq_marital["frequency"] / sum(freq_marital["frequency"]) * 100, 3)
freq_marital["cumulative_percent"] = np.cumsum(freq_marital["percent"])
print("Frequency Table for marital:")
freq_marital

Frequency Table for marital:


,frequency,percent,cumulative_percent
marital,,,
1,5733,62.261,62.261
2,60,0.652,62.913
3,100,1.086,63.999
4,132,1.434,65.433
5,1048,11.381,76.814
6,185,2.009,78.823
7,1950,21.177,100.000


In [59]:
# Frequency table for lfsr94
freq_lfsr94 = data.groupby("lfsr94").agg(frequency=("lfsr94", "size"))
freq_lfsr94["percent"] = round(freq_lfsr94["frequency"] / sum(freq_grade92["frequency"]) * 100, 3)
freq_lfsr94["cumulative_percent"] = np.cumsum(freq_lfsr94["percent"])
print("Frequency Table for lfsr94:")
freq_lfsr94

Frequency Table for lfsr94:


,frequency,percent,cumulative_percent
lfsr94,,,
Employed-Absent,358,3.888,3.888
Employed-At Work,8850,96.112,100.000


In [60]:
# Frequency table for grade92
freq_grade92 = data.groupby("grade92").agg(frequency=("grade92", "size"))
freq_grade92["percent"] = round(freq_grade92["frequency"] / sum(freq_grade92["frequency"]) * 100, 3)
freq_grade92["cumulative_percent"] = np.cumsum(freq_grade92["percent"])
print("Frequency Table for grade92:")
freq_grade92

Frequency Table for grade92:


,frequency,percent,cumulative_percent
grade92,,,
31,3,0.033,0.033
32,2,0.022,0.055
33,2,0.022,0.077
34,2,0.022,0.099
35,2,0.022,0.121
36,2,0.022,0.143
37,14,0.152,0.295
38,9,0.098,0.393
39,609,6.614,7.007


In [63]:
# Frequency table for gender
data["female"] = (data["sex"] == 2) # gender dummy
freq_gender = data.groupby("female").agg(frequency=("female", "size"))
freq_gender["percent"] = round(freq_gender["frequency"] / sum(freq_gender["frequency"]) * 100, 3)
freq_gender["cumulative_percent"] = np.cumsum(freq_gender["percent"])
print("Frequency Table for Gender:")
freq_gender

Frequency Table for Gender:


,frequency,percent,cumulative_percent
female,,,
False,1988,21.59,21.59
True,7220,78.41,100.00


In [72]:
# currently working dummy
data["working"] = (data["lfsr94"] == "Employed-At Work")

# marriage dummies
data["married"] = ((data["marital"] == 1) | (data["marital"] == 2)).astype(int)
data["divorced"] = ((data["marital"] == 3) & (data["marital"] == 5)).astype(int)
data["wirowed"] = (data["marital"] == 4).astype(int)
data["nevermar"] = (data["marital"] == 7).astype(int)

# children dummies
data["child0"] = (data["chldpres"] == 0).astype(int)
data["child1"] = (data["chldpres"] == 1).astype(int)
data["child2"] = (data["chldpres"] == 2).astype(int)
data["child3"] = (data["chldpres"] == 3).astype(int)
data["child4pl"] = (data["chldpres"] >= 4).astype(int)

# wage trasnformaton
data["w"] = data["earnwke"] / data["uhours"] # hourly earnings
data['w'] = pd.to_numeric(data['w'], errors='coerce')
data["lnw"] = np.log(data["w"]) 

# age transformation dummies
data["agesq"] = np.power(data["age"], 2)
data["agecu"] = np.power(data["age"], 3)
data["agequ"] = np.power(data["age"], 4)

# education dummies
data['grade92'] = pd.to_numeric(data['grade92'], errors='coerce')
data["university"] = (data["grade92"] > 40) 
data["ed_MA"] = (data["grade92"] == 44).astype(int)
data["ed_Profess"] = (data["grade92"] == 45).astype(int)
data["ed_Phd"] = (data["grade92"] == 46).astype(int)


In [73]:
data.shape

(9208, 28)

In [74]:
# List of columns to keep
columns_to_keep = [
    "female", "sex",
    "working", "lfsr94",
    "marital", "married", "divorced", "wirowed", "nevermar",
    "chldpres", "child0", "child1", "child2", "child3", "child4pl",
    "earnwke", "uhours", "w", "lnw",
    "age", "agesq", "agecu", "agequ",
    "grade92", "university", "ed_MA", "ed_Profess", "ed_Phd"
]
data = data[columns_to_keep]

data.shape

(9208, 28)

In [75]:
data

,female,sex,working,lfsr94,marital,married,divorced,wirowed,nevermar,chldpres,...,lnw,age,agesq,agecu,agequ,grade92,university,ed_MA,ed_Profess,ed_Phd
10,True,2,True,Employed-At Work,1,1,0,0,0,4,...,3.806662,50,2500,125000,6250000,43,True,0,0,0
33,True,2,True,Employed-At Work,5,0,0,0,0,0,...,3.377246,61,3721,226981,13845841,41,True,0,0,0
37,True,2,False,Employed-Absent,1,1,0,0,0,0,...,3.361971,52,2704,140608,7311616,44,True,1,0,0
42,False,1,True,Employed-At Work,1,1,0,0,0,10,...,3.872800,47,2209,103823,4879681,45,True,0,1,0
71,True,2,True,Employed-At Work,1,1,0,0,0,0,...,2.621039,58,3364,195112,11316496,40,False,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149201,True,2,True,Employed-At Work,3,0,0,0,0,0,...,4.278266,52,2704,140608,7311616,45,True,0,1,0
149242,True,2,True,Employed-At Work,1,1,0,0,0,3,...,3.178054,32,1024,32768,1048576,42,True,0,0,0
149252,True,2,True,Employed-At Work,7,0,0,0,1,0,...,2.088773,20,400,8000,160000,40,False,0,0,0
149290,True,2,True,Employed-At Work,1,1,0,0,0,0,...,2.397895,59,3481,205379,12117361,45,True,0,1,0
